# 07 - Text-to-Query : Extraire des données en langage naturel

**Objectif** : Interroger directement les données d'un dataset via MCP.

**Découverte** : L'API MCP `query_resource_data` accepte une question en langage naturel !

## 1. Setup

In [ ]:
import httpx
import json
import os
from dotenv import load_dotenv

load_dotenv("../.env")

MCP_URL = "https://mcp.data.gouv.fr/mcp"
ALBERT_API_KEY = os.getenv("ALBERT_API_KEY")
ALBERT_API_URL = os.getenv("ALBERT_API_URL", "https://albert.api.etalab.gouv.fr/v1")

print(f"✅ MCP URL: {MCP_URL}")
print(f"✅ Albert API: {'configurée' if ALBERT_API_KEY else 'manquante'}")

## 2. Client MCP (avec gestion de session)

In [ ]:
class MCPClient:
    """Client MCP avec gestion de session."""
    
    def __init__(self, base_url: str):
        self.base_url = base_url
        self.session_id = None
        self._request_id = 0
    
    def _call(self, method: str, params: dict = None) -> dict:
        self._request_id += 1
        payload = {
            "jsonrpc": "2.0",
            "id": self._request_id,
            "method": method,
            "params": params or {}
        }
        
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json, text/event-stream"
        }
        if self.session_id:
            headers["Mcp-Session-Id"] = self.session_id
        
        with httpx.Client(timeout=60, http2=False) as client:
            response = client.post(self.base_url, json=payload, headers=headers)
            if "mcp-session-id" in response.headers:
                self.session_id = response.headers["mcp-session-id"]
            response.raise_for_status()
        
        # Parser SSE
        if "text/event-stream" in response.headers.get("content-type", ""):
            for line in response.text.split("\n"):
                if line.startswith("data:"):
                    data = line[5:].strip()
                    if data:
                        parsed = json.loads(data)
                        if "error" in parsed:
                            raise Exception(f"MCP Error: {parsed['error']}")
                        return parsed.get("result", {})
            return {}
        
        return response.json().get("result", {})
    
    def initialize(self):
        return self._call("initialize", {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {"name": "poc-datagouv", "version": "0.1.0"}
        })
    
    def list_tools(self):
        return self._call("tools/list")
    
    def call_tool(self, name: str, arguments: dict = None) -> str:
        result = self._call("tools/call", {"name": name, "arguments": arguments or {}})
        if result.get("content"):
            texts = [c.get("text", "") for c in result["content"] if c.get("type") == "text"]
            return "\n".join(texts)
        return str(result)

# Initialiser
mcp = MCPClient(MCP_URL)
mcp.initialize()
print(f"✅ MCP connecté (session: {mcp.session_id[:8]}...)")

## 3. Explorer les tools disponibles

In [ ]:
tools = mcp.list_tools()

print(f"📦 {len(tools.get('tools', []))} tools disponibles:\n")

for tool in tools.get("tools", []):
    print(f"• {tool['name']}")
    print(f"  {tool.get('description', '')[:80]}...")
    print()

In [ ]:
# Focus sur query_resource_data
query_tool = next((t for t in tools.get("tools", []) if t["name"] == "query_resource_data"), None)

if query_tool:
    print("=== query_resource_data ===")
    print(f"\nDescription:\n{query_tool.get('description')}")
    print(f"\nParamètres:\n{json.dumps(query_tool.get('inputSchema', {}), indent=2)}")

## 4. Test avec un vrai dataset

Dataset : **Bornes de recharge électrique - Toulouse**

In [ ]:
# Recherche de datasets
result = mcp.call_tool("search_datasets", {
    "query": "bornes recharge électrique",
    "page_size": 3
})
print(result)

In [ ]:
# Dataset Toulouse
DATASET_ID = "5fcb06852833ccfe8d43096a"

result = mcp.call_tool("list_dataset_resources", {"dataset_id": DATASET_ID})
print(result)

In [ ]:
# Ressource CSV
RESOURCE_ID = "9e75a85e-255e-4f4d-8bf5-713587c1ba5e"

result = mcp.call_tool("get_resource_info", {"resource_id": RESOURCE_ID})
print(result)

## 5. query_resource_data en action

In [ ]:
# Test 1 : Question simple
result = mcp.call_tool("query_resource_data", {
    "resource_id": RESOURCE_ID,
    "question": "Combien de bornes y a-t-il ?",
    "page_size": 3
})
print(result[:2500])

In [ ]:
# Test 2 : Question sur les communes
result = mcp.call_tool("query_resource_data", {
    "resource_id": RESOURCE_ID,
    "question": "Quelles sont les bornes à Blagnac ?",
    "page_size": 20
})
print(result[:3000])

## 6. Observation importante

**Le paramètre `question` ne filtre PAS les données !**

Les cellules 13 et 14 retournent les mêmes lignes (Row 1: Aussonne, Row 2: Balma, Row 3: Blagnac...) 
quelle que soit la question posée. Seul le `page_size` change le nombre de résultats.

### Comportement réel de l'API :
```
query_resource_data(question="...", resource_id="...", page_size=N)
                         ↓
           Retourne TOUJOURS les N premières lignes
           (la question est ignorée côté serveur)
```

### Le filtrage se fait côté LLM :
```
Données brutes → LLM + Question → Réponse filtrée
```

C'est l'architecture **RAG classique** : l'API fournit le contexte, le LLM interprète.

## 7. Pipeline complet avec LLM

In [ ]:
def chat_completion(messages: list) -> str:
    """Appel au LLM Albert."""
    with httpx.Client(timeout=60) as client:
        response = client.post(
            f"{ALBERT_API_URL}/chat/completions",
            headers={"Authorization": f"Bearer {ALBERT_API_KEY}", "Content-Type": "application/json"},
            json={
                "model": "mistralai/Mistral-Small-3.2-24B-Instruct-2506",
                "messages": messages,
                "temperature": 0.2,
                "max_tokens": 1024
            }
        )
        response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

print("✅ Fonction LLM prête")

In [ ]:
def query_and_answer(resource_id: str, question: str, page_size: int = 50, max_chars: int = 12000) -> str:
    """
    Pipeline complet : question → données MCP → réponse LLM.
    
    Args:
        resource_id: ID de la ressource MCP
        question: Question en langage naturel
        page_size: Nombre de lignes à récupérer
        max_chars: Limite de caractères envoyés au LLM (défaut: 12000 ≈ 15-20 lignes)
    """
    # 1. Récupérer les données
    print(f"🔍 Interrogation de la ressource (page_size={page_size})...")
    data = mcp.call_tool("query_resource_data", {
        "resource_id": resource_id,
        "question": question,
        "page_size": page_size
    })
    
    # 2. Demander au LLM d'analyser
    data_truncated = data[:max_chars]
    print(f"🤖 Analyse par le LLM ({len(data_truncated):,} chars sur {len(data):,})...")
    
    messages = [
        {"role": "system", "content": """Tu es un assistant qui analyse des données publiques.
À partir des données fournies, réponds à la question de l'utilisateur.
Sois précis et cite des chiffres/exemples concrets des données.
Note : tu reçois un échantillon des données, pas la totalité."""},
        {"role": "user", "content": f"""Données extraites de data.gouv.fr :

{data_truncated}

---

Question : {question}"""}
    ]
    
    return chat_completion(messages)

print("✅ Pipeline prêt (max_chars=12000 par défaut)")

In [ ]:
# Test du pipeline complet (avec plus de données)
question = "Combien de bornes de recharge y a-t-il au total et dans quelles communes ?"

response = query_and_answer(RESOURCE_ID, question, page_size=112)  # Toutes les lignes
print(f"\n💬 Réponse :\n{response}")

In [ ]:
# Autre test avec plus de contexte
question = "Quelle est la puissance moyenne des bornes et quels types de connecteurs sont disponibles ?"

response = query_and_answer(RESOURCE_ID, question, page_size=112, max_chars=15000)
print(f"\n💬 Réponse :\n{response}")

## 8. Solution : download_and_parse_resource

Le serveur MCP ne formate que 3 lignes dans `query_resource_data`.
Pour analyser toutes les données, utilisons `download_and_parse_resource`.

In [ ]:
# Télécharger et parser toutes les données
full_data = mcp.call_tool("download_and_parse_resource", {
    "resource_id": RESOURCE_ID
})

print(f"📊 Données complètes : {len(full_data):,} caractères")
print(full_data[:3000])

In [ ]:
def query_full_data(resource_id: str, question: str, max_chars: int = 15000) -> str:
    """
    Pipeline avec données complètes via download_and_parse_resource.
    """
    print(f"🔍 Téléchargement des données complètes...")
    data = mcp.call_tool("download_and_parse_resource", {"resource_id": resource_id})
    
    data_truncated = data[:max_chars]
    print(f"🤖 Analyse par le LLM ({len(data_truncated):,} chars sur {len(data):,})...")
    
    messages = [
        {"role": "system", "content": """Tu es un assistant qui analyse des données publiques.
À partir des données fournies, réponds à la question de l'utilisateur.
Sois précis, fais des calculs si nécessaire, et cite des exemples concrets."""},
        {"role": "user", "content": f"""Données extraites de data.gouv.fr :

{data_truncated}

---

Question : {question}"""}
    ]
    
    return chat_completion(messages)

print("✅ Fonction query_full_data prête")

In [ ]:
# Test avec données complètes
question = "Quelle est la puissance moyenne des bornes et quels types de connecteurs sont disponibles ?"

response = query_full_data(RESOURCE_ID, question)
print(f"\n💬 Réponse :\n{response}")

In [ ]:
# Test : question sur les communes
question = "Combien de bornes y a-t-il dans chaque commune ? Donne-moi le top 5."

response = query_full_data(RESOURCE_ID, question, max_chars=25000)
print(f"\n💬 Réponse :\n{response}")

## 9. Alternative : Téléchargement direct du CSV

Le serveur MCP limite les données retournées (~3 lignes).
Solution : télécharger le CSV directement et l'analyser localement.

In [ ]:
import pandas as pd

# URL du CSV (récupérée via MCP get_resource_info)
CSV_URL = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/bornes-recharge-electrique/exports/csv?use_labels=true"

# Télécharger directement
df_bornes = pd.read_csv(CSV_URL, sep=";")
print(f"✅ {len(df_bornes)} lignes chargées")
print(f"📊 Colonnes : {list(df_bornes.columns)}")
df_bornes.head()

In [ ]:
# Analyse réelle des données
print("📊 Statistiques réelles :\n")

# Puissance moyenne
print(f"⚡ Puissance moyenne : {df_bornes['puissance_prise'].mean():.1f} kW")
print(f"   Min: {df_bornes['puissance_prise'].min()} kW, Max: {df_bornes['puissance_prise'].max()} kW")

# Connecteurs
print(f"\n🔌 Types de connecteurs :")
connecteurs = df_bornes['connecteur_disponible_prise'].value_counts()
for conn, count in connecteurs.items():
    print(f"   {conn}: {count} bornes")

# Top communes
print(f"\n🏘️ Top 5 communes :")
communes = df_bornes['commune'].value_counts().head(5)
for commune, count in communes.items():
    print(f"   {commune}: {count} bornes")

In [ ]:
def query_csv_with_llm(df: pd.DataFrame, question: str, sample_size: int = 50) -> str:
    """
    Envoie un échantillon du DataFrame au LLM pour analyse.
    """
    # Convertir en texte (échantillon)
    csv_sample = df.head(sample_size).to_csv(index=False)
    stats = f"Total: {len(df)} lignes, Colonnes: {list(df.columns)}"
    
    print(f"🤖 Analyse par le LLM ({len(csv_sample):,} chars, {sample_size} lignes sur {len(df)})...")
    
    messages = [
        {"role": "system", "content": """Tu es un assistant qui analyse des données CSV.
Réponds à la question en te basant sur l'échantillon fourni.
Fais des calculs précis et cite des exemples concrets."""},
        {"role": "user", "content": f"""Données CSV ({stats}) :

{csv_sample}

---

Question : {question}"""}
    ]
    
    return chat_completion(messages)

# Test
question = "Quelle est la puissance moyenne des bornes ? Quels types de connecteurs sont disponibles ?"
response = query_csv_with_llm(df_bornes, question, sample_size=112)  # Toutes les lignes
print(f"\n💬 Réponse LLM :\n{response}")